In [1]:
import numpy as np

import holoviews as hv
import panel as pn
hv.extension('bokeh', logo=False)
pn.extension()

In [2]:
# Draw ngons with possible shading of the interior

def ngon_data( x=0,y=0, specs=[(1,1)], orientation=0., arc=(0,2*np.pi), npoints=3 ):
    ''' compute coordinate locations for n-gons with vertices on an ellipse

    Args:
        x           : x coordinate of n-gon center
        y           : y coordinate of n-gon center
        specs       : array of (a,b) lengths of principal axes of the ellipse(s)
        orientation : inclination of the principal axis of the ellipse in radians
        arc         : (a1,a2) angles in radians of the subtended arc of the n-gon vertices
        npoints     : number of vertices

    Notes:
        | since the resulting paths are closed, the computation yields npoints+1 vertices;
        | thus, arcs that do not correspond to a full circle result in an extra vertex
    '''
    t_vals = np.linspace(*arc,npoints+1)
    s = np.sin(t_vals)
    c = np.cos(t_vals)
    rot  = np.array([[np.cos(orientation), -np.sin(orientation)],
                     [np.sin(orientation),  np.cos(orientation)]])
    gons = []
    for (a,b) in specs:
        data = np.array(list(zip(a*c,b*s)))
        gons.append( np.tensordot(rot, data.T, axes=[1,0]).T+np.array([x,y]))

    return gons

def ngon(x=0,y=0, spec=(1,1), orientation=0., arc=(0,2*np.pi), npoints=40,\
         line_color='k', line_alpha=1., color='k', alpha=0., label=''):
    e = ngon_data(x,y,specs=[spec], arc=arc,orientation=orientation,npoints=npoints)

    return hv.Polygons(e, label=label).opts( line_color=line_color,line_alpha=line_alpha, color=color, alpha=alpha)

def circle(x=0, y=0, radius=1, arc=(0,2*np.pi), npoints=40,\
         line_color='k', line_alpha=1., color='k', alpha=0., label=''):
    e = ngon_data(x,y,specs=[(radius,radius)], arc=(0,2*np.pi),orientation=0.,npoints=npoints)
    return hv.Polygons(e, label=label).opts( line_color=line_color,line_alpha=line_alpha, color=color, alpha=alpha, aspect="equal")

<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Greshgorin Circles</strong>
</div>

# 1. Basic Definitions and Theorems

The **Greshgorin Circle Theorem** may be used to bound the spectrum of a matrix.

## 1.1 Greshgorin Disks

Let $A = \left( a_{i j} \right)$ be a matrix in $\mathscr{C}^{N \times N}$. 


**Greshgorin disks** are disks in the complex plane centered on the diagonal entries $a_{i i}$ with radius $R_i = \sum_{j \ne i} {\left| a_{i j} \right|}$,<br> $\qquad$ i.e., the sum of the absolute values of the non-diagonal entries
in the $i^{th}$ row.

Consider the following two examples:

$A = \left(\begin{array}{rrr} 0 & 1 & \frac{1}{2} \\ \frac{1}{4} & 5 & -\frac{1}{4} \\ -1 & 1 & -1\end{array}\right)\quad$
for which we find $\left\{ \begin{align}
\left(a_{1 1}, R_1\right) &=\; ( 0, \frac{3}{2})\\
\left(a_{2 2}, R_2\right) &=\; ( 5, \frac{1}{2})\\
\left(a_{3 3}, R_3\right) &=\; (-1, 2)\\
\end{align}\right.$

and

$B = \left(\begin{array}{rrr} 0 & \frac{1}{4} & \frac{1}{2} \\ \frac{1+i}{2} & 4-i & 0 \\ -1 & i & -i\end{array}\right)\quad$
for which we find $\left\{ \begin{align}
\left(a_{1 1}, R_1\right) &=\; ( 0, \frac{3}{4})\\
\left(a_{2 2}, R_2\right) &=\; ( 4-i, \frac{\sqrt{2}}{2})\\
\left(a_{3 3}, R_3\right) &=\; (-i, 2)\\
\end{align}\right.$

In [3]:
def greshgorin_disks(A, color="blue", alpha=0.1):
    centers     = [(c.real, c.imag) for c in np.diag(A)]
    radii       = np.sum(np.abs(A), axis=1)-np.diag( abs(A))
    circles     = [circle( centers[i][0], centers[i][1], radii[i],
                           line_color=color, color=color, alpha=alpha) for i in range(len(centers))] 
    return centers, radii, circles

def show_greshgorin_disks( A, nm, show_axes=True, color="blue", alpha=0.1 ):
    centers, radii, circles = greshgorin_disks(A, color=color, alpha=alpha)

    eigenvals   = [(e.real, e.imag) for e in np.linalg.eigvals(A)]

    ha = hv.Overlay( [hv.Points( centers,   label="center"    ).opts(color='blue', size=6, aspect="equal"),
                      hv.Points( eigenvals, label="eigenvalue").opts(color='red',  size=6, aspect="equal")]+\
                     circles )
    if show_axes:
        ha = ha*hv.HLine(0).opts(line_width=0.6, color='k')\
               *hv.VLine(0).opts(line_width=0.6, color='k')
    return ha.opts(title="Greshgorin Circles for Matrix "+nm, xlabel="Re(x)", ylabel="Im(x)", legend_position="top")

In [4]:
A = np.array([[0,1,1/2],[1//4,5,-1//4],[-1,1,-1]])
B = np.array([[0,1/4,1/2],[(1+1j)/2,4-1j, 0],[-1,1j,0-1j]])

h=\
show_greshgorin_disks(A, "A").opts(frame_width=350)+\
show_greshgorin_disks(B, "B").opts(frame_width=350)
h

:Layout
   .Overlay.I  :Overlay
      .Points.Center     :Points   [x,y]
      .Points.Eigenvalue :Points   [x,y]
      .Polygons.I        :Polygons   [x,y]
      .Polygons.II       :Polygons   [x,y]
      .Polygons.III      :Polygons   [x,y]
      .HLine.I           :HLine   [x,y]
      .VLine.I           :VLine   [x,y]
   .Overlay.II :Overlay
      .Points.Center     :Points   [x,y]
      .Points.Eigenvalue :Points   [x,y]
      .Polygons.I        :Polygons   [x,y]
      .Polygons.II       :Polygons   [x,y]
      .Polygons.III      :Polygons   [x,y]
      .HLine.I           :HLine   [x,y]
      .VLine.I           :VLine   [x,y]

## 1.2 Greshgorin Circle Theorem

<div style="float:left;background-color:#F2F5A9;color:black;padding-right:0.3cm;">

**Theorem:** Every eigenvalue of $A$ lies in at least on of the Greshgorin disks of $A$.
</div>

**Proof:** Let $(\lambda, x)$ be an eigenpair for $A$, and let $i$ be the index of the largest absolute value in $x$.
<br>$\qquad$ Since $A x = \lambda x$, the $i^{th}$ entry is given by $\sum_j{a_{i j} x_j} = \lambda x_i \Leftrightarrow \sum_{j\ne i}{a_{i j} x_j} = \left(\lambda - a_{i i} \right)x_i $.

$\qquad$ Using the triangle inequality and the fact that $\left| \frac{x_j}{x_i} \right| \le 1$ given how we picked $i$, we find
$$\left| \lambda - a_{i i} \right| = \left| \sum_{j \ne i} \frac{ a_{ i j} x_j}{x_i}\right| \le \sum_{j \ne i} \left| a_{ i j}\right| = R_i$$


**Remarks:**
* The theorem **does not claim** that there is one disc for each eigenvalue
* Since $A$ and $A^t$ have the same eigenvalues, the eigenvalues must also lie <br>$\quad$
within the Greshgorin disks corresponding to the columns of $A$.

In [5]:
_,_,circles_A = greshgorin_disks(A.T, color='cyan')
_,_,circles_B = greshgorin_disks(B.T, color='cyan')

(hv.Overlay(circles_A)*show_greshgorin_disks(A, "A"))\
   .opts(frame_width=350, legend_position='top', title="Greshgorin Circles for Matrix A")+\
(hv.Overlay(circles_B)*show_greshgorin_disks(B, "B"))\
   .opts(frame_width=350, legend_position='top', title="Greshgorin Circles for Matrix B")

:Layout
   .Overlay.I  :Overlay
      .Polygons.I        :Polygons   [x,y]
      .Polygons.II       :Polygons   [x,y]
      .Polygons.III      :Polygons   [x,y]
      .Points.Center     :Points   [x,y]
      .Points.Eigenvalue :Points   [x,y]
      .Polygons.IV       :Polygons   [x,y]
      .Polygons.V        :Polygons   [x,y]
      .Polygons.VI       :Polygons   [x,y]
      .HLine.I           :HLine   [x,y]
      .VLine.I           :VLine   [x,y]
   .Overlay.II :Overlay
      .Polygons.I        :Polygons   [x,y]
      .Polygons.II       :Polygons   [x,y]
      .Polygons.III      :Polygons   [x,y]
      .Points.Center     :Points   [x,y]
      .Points.Eigenvalue :Points   [x,y]
      .Polygons.IV       :Polygons   [x,y]
      .Polygons.V        :Polygons   [x,y]
      .Polygons.VI       :Polygons   [x,y]
      .HLine.I           :HLine   [x,y]
      .VLine.I           :VLine   [x,y]

$\therefore\quad$ **The eigenvalues must lie in the intersection of the blue and cyan disks.**

## 1.3 Strengthening the Theorem

<div style="float:left;background-color:#F2F5A9;color:black;padding-right:0.3cm;">

**Theorem:** If the union of $k$ discs is disjoint from the union of the remaining $n − k$ discs<br>
$\qquad$ then the former union contains exactly $k$ and the latter $n − k$ eigenvalues of $A$,<br>
$\qquad$ where the eigenvalues are counted with their algebraic multiplicities.
</div>

> In the previous example, there are two overlapping disks that contain 2 eigenvalues, and the third non-overlapping disk contains a single eigenvalue.

**Remark:** Greshgorin disks do not necessarily contain eigenvalues.

In the following two examples, the smaller disk does not contain an eigenvalue!

In [6]:
A = np.array([[0,4],[1,0]])
B = np.array([[1,1],[-2,-1]])

h=\
show_greshgorin_disks(A, "A").opts(frame_height=250)+\
show_greshgorin_disks(B, "B").opts(frame_height=250)
h

:Layout
   .Overlay.I  :Overlay
      .Points.Center     :Points   [x,y]
      .Points.Eigenvalue :Points   [x,y]
      .Polygons.I        :Polygons   [x,y]
      .Polygons.II       :Polygons   [x,y]
      .HLine.I           :HLine   [x,y]
      .VLine.I           :VLine   [x,y]
   .Overlay.II :Overlay
      .Points.Center     :Points   [x,y]
      .Points.Eigenvalue :Points   [x,y]
      .Polygons.I        :Polygons   [x,y]
      .Polygons.II       :Polygons   [x,y]
      .HLine.I           :HLine   [x,y]
      .VLine.I           :VLine   [x,y]

____
**Remark:** Eigenvalues may occur at the boundaries of the disks.

The matrix $A =\begin{pmatrix} 0 & 1 & 1 & 0 \\ 1 & 0 & 0 & 1 \\ 1 & 0 & 0 & 1 \\ 0 & 1 & 1 & 0 \end{pmatrix}$
has eigenvalues $\left\{ -2,0,0, 1\right\}$.

Each of the Greshgorin disks is a disk of radius 2 centered at the origin.

In [11]:
A = np.array([[0,1,1,0],[1,0,0,1],[1,0,0,1],[0,1,1,0]])
show_greshgorin_disks(A, "A").opts(frame_width=210)

:Overlay
   .Points.Center     :Points   [x,y]
   .Points.Eigenvalue :Points   [x,y]
   .Polygons.I        :Polygons   [x,y]
   .Polygons.II       :Polygons   [x,y]
   .Polygons.III      :Polygons   [x,y]
   .Polygons.IV       :Polygons   [x,y]
   .HLine.I           :HLine   [x,y]
   .VLine.I           :VLine   [x,y]